# Step 0: Imports

In [42]:
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups # We use the 20 news groups text dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
import pandas as pd

# Step 1: Fetching data and preprocessing

In [43]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [63]:
print(len(newsgroups_train.target_names))

20


In [45]:
# Split into smaller training sets in percentage
percentage = 0.2
split_index = int(len(newsgroups_train.data) * percentage)
train_data_small = newsgroups_train.data[:split_index]
train_targets_small = newsgroups_train.target[:split_index]

In [46]:
print(train_data_small[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [47]:
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))

# Initialize structures for the preprocessed corpus
filtered_train = [[] for _ in range(len(train_data_small))]  # Preprocessed articles
flattened_train = []  # A single list of all words in the corpus

# Tokenizing and removing stopwords
print("Processing Articles")
for i, article in tqdm(enumerate(train_data_small), total=len(train_data_small)):
    word_tokens = word_tokenize(article)  # Tokenize article
    # Remove stop words and add to both filtered_train and flattened_train
    filtered_words = [w.lower() for w in word_tokens if w.lower() not in stop_words]
    filtered_train[i] = filtered_words
    flattened_train.extend(filtered_words)

# Create a vocabulary mapping
unique_words = sorted(set(flattened_train))  # Get unique words
word_to_id = {word: idx for idx, word in enumerate(unique_words)}  # Map word to ID
id_to_word = {idx: word for word, idx in word_to_id.items()}  # Reverse mapping

# Map the filtered articles to integer IDs
int_corpus = [[word_to_id[word] for word in article] for article in filtered_train]

# Display mappings and a small example
print(f"Total unique words: {len(unique_words)}")
print("Word-to-ID mapping example:", {k: word_to_id[k] for k in list(word_to_id)[:5]})
print("Integer Corpus example (first article):", int_corpus[0][100:110])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Processing Articles


100%|██████████| 2262/2262 [00:07<00:00, 309.74it/s]


Total unique words: 70841
Word-to-ID mapping example: {'!': 0, '#': 1, '$': 2, '%': 3, '&': 4}
Integer Corpus example (first article): []


In [48]:
word_count = Counter(flattened_train)

In [49]:
# Extract low-frequency words (occurrence <= 10) into a set
low_frequency_words = {word for word, count in word_count.items() if count <= 10}

In [50]:
# Filter articles efficiently using set operations
corpus_hf = []
for article in tqdm(int_corpus, desc="Removing LF words"):
    article_set = set(article)
    filtered_article = list(article_set - low_frequency_words)
    corpus_hf.append(filtered_article)



Removing LF words: 100%|██████████| 2262/2262 [00:00<00:00, 16664.85it/s]


In [51]:
flattened_train = [word for word in flattened_train if word not in low_frequency_words]

# Step 2: Gibbs sampling

In [54]:
from collections import defaultdict

def lda_gibbs_sampling(corpus, K, alpha, beta, iterations):
    """
    Implements Collapsed Gibbs Sampling for LDA.

    :param corpus: List of lists, where each inner list contains word IDs in a document.
    :param K: Number of topics.
    :param alpha: Dirichlet prior for document-topic distribution.
    :param beta: Dirichlet prior for topic-word distribution.
    :param iterations: Number of Gibbs sampling iterations.
    :return: Topic assignments, document-topic counts, topic-word counts, topic totals.
    """
    # Initialize variables
    D = len(corpus)  # Number of documents
    V = max(max(doc) for doc in corpus) + 1  # Vocabulary size (assumes word IDs are 0-indexed)

    # Count matrices
    ndk = np.zeros((D, K))  # Document-topic counts
    nkw = np.zeros((K, V))  # Topic-word counts
    nk = np.zeros(K)        # Total words in each topic

    # Topic assignments for each word
    z = []  # Topic assignment for each word in corpus
    for d, doc in enumerate(corpus):
        doc_topics = []
        for word in doc:
            topic = np.random.randint(K)  # Randomly assign a topic
            doc_topics.append(topic)
            ndk[d, topic] += 1
            nkw[topic, word] += 1
            nk[topic] += 1
        z.append(doc_topics)

    # Gibbs sampling
    for _ in tqdm(range(iterations)):
        for d, doc in enumerate(corpus):
            for i, word in enumerate(doc):
                current_topic = z[d][i]

                # Decrement counts
                ndk[d, current_topic] -= 1
                nkw[current_topic, word] -= 1
                nk[current_topic] -= 1

                # Compute topic probabilities
                topic_probs = (ndk[d] + alpha) * (nkw[:, word] + beta) / (nk + beta * V)
                topic_probs /= np.sum(topic_probs)  # Normalize

                # Sample new topic
                new_topic = np.random.choice(K, p=topic_probs)
                z[d][i] = new_topic

                # Increment counts
                ndk[d, new_topic] += 1
                nkw[new_topic, word] += 1
                nk[new_topic] += 1

    return z, ndk, nkw, nk

In [ ]:
# Initialize data
corpus = corpus_hf.copy()
targets = train_targets_small.copy()
topics = newsgroups_train.target_names.copy()

K = len(topics)

# First parameter combo
z, ndk, nkw, nk = lda_gibbs_sampling(corpus, alpha = 0.1, beta = 0.1, K=K, iterations=50)

 60%|██████    | 30/50 [06:24<04:12, 12.62s/it]

In [62]:
print(ndk.shape, nkw.shape, nk.shape)

(2262, 20) (20, 70841) (20,)


In [82]:
# Find the best matching topic for each label
topic_to_target = {}
for t in range(K):
    most_common_target = np.bincount([targets[i] for i in range(len(corpus)) if np.argmax(ndk[i]) == t]).argmax()
    topic_to_target[t] = most_common_target

# Map predicted topics to target labels
predicted_targets = [topic_to_target[np.argmax(doc)] for doc in ndk]
accuracy = np.mean(predicted_targets == targets)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 18.26%


In [83]:
predicted_targets = [np.argmax(i) for i in ndk]
accuracy = np.mean(predicted_targets == targets)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 6.19%
